In [1]:
from hdx.utilities.easy_logging import setup_logging
from hdx.hdx_configuration import Configuration
from hdx.data.dataset import Dataset
import numpy as np
import pandas as pd

In [2]:
setup_logging()

No logging configuration parameter. Using default.
Loading logging configuration from: c:\users\vanis\anaconda3\lib\site-packages\hdx\utilities\logging_configuration.yml


In [3]:
Configuration.create(hdx_site='prod', user_agent='A_Quick_Example')

INFO - 2019-03-04 22:50:32 - hdx.hdx_configuration - No HDX base configuration parameter. Using default base configuration file: c:\users\vanis\anaconda3\lib\site-packages\hdx\hdx_base_configuration.yml.
INFO - 2019-03-04 22:50:32 - hdx.hdx_configuration - Loading HDX base configuration from: c:\users\vanis\anaconda3\lib\site-packages\hdx\hdx_base_configuration.yml
INFO - 2019-03-04 22:50:32 - hdx.hdx_configuration - No HDX configuration parameter and no configuration file at default path: C:\Users\vanis\.hdx_configuration.yml.
INFO - 2019-03-04 22:50:32 - hdx.hdx_configuration - Read only access to HDX: False


'https://data.humdata.org/'

In [4]:
from hdx.facades.simple import facade

No logging configuration parameter. Using default.
Loading logging configuration from: c:\users\vanis\anaconda3\lib\site-packages\hdx\utilities\logging_configuration.yml


In [5]:
datasets = Dataset.get_all_dataset_names()
print(len(datasets)) #Total number of datasets obtained using the API


12148


In [6]:
#Script to clean the datasets list and return the list of datasets that have at least 1
#resource of the required file type(s).
#Currently very naive and inefficient
def clean(datasets, file_types=[]):
    result = []
    index = 0
    while (index < len(datasets)):
        temp_dataset = Dataset.read_from_hdx(datasets[index])
        temp_dataset.separate_resources() #In order to use dataset.resources
        if (len(temp_dataset.resources) > 0):
            if (len(file_types) > 0):
                if (not set(temp_dataset.get_filetypes()).isdisjoint(file_types)): #To check if elements of the 2 lists of file_types overlap.
                    result.append(datasets[index])
            else :
                result.append(datasets[index])
        index += 1
    return result

In [7]:
def dload_dset(dataset_1,dload_path,row_limit=10,file_type='CSV') :
    if (file_type == None):
        url, path = dataset_1.resources[0].download(dload_path)
        pandas_dataset = pd.read_csv(path)
    else :
        if (file_type not in dataset_1.get_filetypes()):
            url = '1'
            path = '2'
            pandas_dataset ='9'
            return 'Error: Required file type not in dataset OR dataset does not contain any resources.'
        else :
            url, path = dataset_1.resources[dataset_1.get_filetypes().index(file_type)].download(dload_path)
            pandas_dataset = pd.read_csv(path, encoding='latin-1')
    pandas_dataset = pandas_dataset.head(row_limit)
    return pandas_dataset

In [8]:
#hdx_dset_scraper downloads a certain number (limit) of datasets from HDX to the given 
#download path of the given file_type (currently only 'CSV' is supported due to pd.read_csv). To use, 
#import both hdx_dset_scraper and dload_dset.
def hdx_dset_scraper(dload_path,limit=1000000000,download=True,file_type='CSV'):
    datasets = Dataset.get_all_dataset_names()
    if (limit > len(datasets)):
        limit = len(datasets)
    dset_dict = {}
    index = 0
    while (limit > index) :
        dataset_1 = Dataset.read_from_hdx(datasets[index]) 
        dataset_1.separate_resources()
        if (file_type not in dataset_1.get_filetypes()) :
            index += 1
            limit += 1
        else :
            dset_dict.update({datasets[index] : dload_dset(dataset_1,dload_path)})
            index += 1
    return dset_dict

In [ ]:
dict = hdx_dset_scraper('Datasets',limit=10)
print(dict)

In [ ]:
def test_clean(datasets):
    for i in datasets[0:10]:
        print(Dataset.read_from_hdx(i).get_filetypes(),i)
test_clean(datasets)

In [9]:
dataset_1 = Dataset.read_from_hdx(datasets[datasets.index('160523-ocha-4w-round-2')]) 
dataset_1.separate_resources() 
print(len(dataset_1.resources))
print(dataset_1.get_filetypes()) 
print(dataset_1.resources[0].get_resource_views())

5
['XLSX', 'CSV', 'GeoJSON', 'topojson', 'JSON']
[{'description': '', 'title': 'Data Explorer', 'resource_id': '1e51403e-0044-4b47-8f1c-1268c759f0df', 'view_type': 'recline_view', 'id': '21e4e30f-6d79-4e5b-b19d-3d3c47fe797b', 'package_id': 'bdc7d663-7332-4d2d-a5ca-6fc52cae882e'}]


In [12]:
url, path = dataset_1.resources[1].download('Datasets') #Substitute Datasets with your file download path.
print('Resource URL %s downloaded to %s' % (url, path))

Resource URL http://data.humdata.org/dataset/bdc7d663-7332-4d2d-a5ca-6fc52cae882e/resource/b6f2ab6d-7618-4768-8ef0-3bcd9ea34827/download/160523ecuador_4w_hdx.csv downloaded to Datasets\160523ECUADOR_4W_HDX.csv.CSV


In [13]:
pandas_dataset = pd.read_csv(path)
pandas_dataset

,Organizacion,Sector,Provincia,pcode,Canton,pcode2,Estado
0,Arche noVa,"Agua, Saneamiento e Higiene",Esmeraldas,EC08,Muisne,EC0803,En ejecución
1,Acción Contra el Hambre,Recuperación temprana,Esmeraldas,EC08,Muisne,EC0803,Finalizado
2,Acción Contra el Hambre,Recuperación temprana,Manabí,EC13,Pedernales,EC1317,Finalizado
3,Arche noVa,"Agua, Saneamiento e Higiene",Esmeraldas,EC08,Muisne,EC0803,En ejecución
4,Arche noVa,"Agua, Saneamiento e Higiene",Manabí,EC13,San Vicente,EC1322,En ejecución
5,Arche noVa,"Agua, Saneamiento e Higiene",Manabí,EC13,Jama,EC1320,En ejecución
6,Catholic Relief Services,"Agua, Saneamiento e Higiene",Esmeraldas,EC08,Muisne,EC0803,En ejecución
7,Catholic Relief Services,"Agua, Saneamiento e Higiene",Manabí,EC13,Pedernales,EC1317,En ejecución
8,Catholic Relief Services,"Agua, Saneamiento e Higiene",Manabí,EC13,Jama,EC1320,En ejecución
9,Cooperación Técnica Belga,"Agua, Saneamiento e Higiene",Manabí,EC13,Jipijapa,EC1306,En ejecución


In [ ]:
pandas_dataset.head(5)